# NCBI Datasets - CSHL (11/02/2021)

### Table of contents
* [Part I: Accessing genomes](#Part-I)
* [Part II: Accessing genes](#Part-II)
* [Part III: Accessing orthologs](#Part-III)
* [Part IV: Building a BLAST database and creating a phylogenetic tree](#Part-IV)
* [Part V: Downloading large datasets (dehydration/rehydration) and `dataformat`](#Part-V)

### Important resources
- Etherpad: https://etherpad.wikimedia.org/p/CSHL_Datasets_Workshop_2021
- Github: https://github.com/ncbi/datasets/tree/workshop-cshl-2021/training/cshl-2021
- NCBI datasets: https://www.ncbi.nlm.nih.gov/datasets/
- jq cheat sheet: https://github.com/ncbi/datasets/blob/workshop-cshl-2021/training/cshl-2021/jq_cheatsheet.md

## Before we start... What is a jupyter notebook?

Jupyter Notebooks are a web-based approach to interactive code. A single notebook (the file you are currently reading) is composed of many "cells" which can contain either text, or code. To navigate between cells, either click, or use the arrow keys on your keyboard.

A text cell will look like... well... this! While a code cell will look something like what you see below. To run the code inside a code cell, click on it, then click the "Run" button at the top of the screen. Try it on the code cell below!

In [1]:
#This is a code cell
print('You ran the code cell!')

You ran the code cell!


If it worked, you should have seen text pop up underneath the cell saying `You ran the code cell!`. Note the `In [1]:` that appeared next to the cell. This tells you the order you have run code cells throughout the notebook. The next time you run a code cell, it will say `In [2]:`, then `In [3]:` and so on... This will help you know if/when code has been run.

The remainder of the notebook below has been pre-built by the workshop organizer. You will not need to create any new cells, and you will be explicitly told if/when to execute a code cell.

The code in this workshop is either Bash (i.e., terminal commands) or Python. Bash commands are prefixed with `!` or the cells have the notation `%%bash` at the top., while Python commands are not. If you are not familiar with code, don't feel pressured to interpret it very deeply. Descriptions of each code block will be provided!

(Jupyter Notebook explanation by Cooper Park at the workshop on [Finding and Analyzing Metagenomic Data](https://www.nlm.nih.gov/oet/ed/ncbi/2021_10_meta.html))

## Case study: Elmo loves ants

Elmo is a graduate student at the Via Sesamum University. As part of his Ph.D. project, he studies Panamanian leaf cutter ants (genus *Acromyrmex*, family Formicidae) and how variation in the gene *orco* (**o**dorant **r**eceptor **co**receptor) affects the colonies of this genus.

(here's the [link](https://www.ncbi.nlm.nih.gov/labs/pmc/articles/PMC5556950/) to a cool paper talking about this gene in ants of the species *Ooceraea biroi*).

<img src="./images/ants.png" alt="image"/>

Elmo will use `datasets` to help him gather the existing genomic resources from NCBI. He will:

- download all available genomes for the genus *Acromyrmex*
- download the *orco* gene from the *Acromyrmex* reference genome
- download the ortholog set for this gene for all ants (Formicidae)

In addition, he will also do the following tasks:
- Create a custom BLAST database with the Panamanian leaf cutter ants genomes 
- BLAST the gene *orco* against the database
- Multiple sequence alignment of the BLAST results and the ortholog gene sequences
- Build a phylogenetic tree using fastTree


### How is `datasets` organized?

[NCBI datasets](https://www.ncbi.nlm.nih.gov/datasets/docs/v1/quickstarts/command-line-tools/) is a command line tool that allows users to download data packages (data + metadata) or look at metadata summaries for genomes, RefSeq annotated genes, curated ortholog sets and SARS-Cov-2 virus sequences and proteins. The program follows a hierarchy that makes it easier for users to select exact which options they would like to use. In addition to the program commands, additional flags are available for filtering the results. We will go over those during this tutorial.
<img src="./images/datasets_horizontal.drawio.png" alt="datasets" style="width: 600px;"/>

In addition to `datasets`, we will be using `jq` (JSON parser) to take a look at the metadata information. Our metadata reports are almost all in JSON or [JSON Lines](https://jsonlines.org/) format. We put together a [jq cheat sheet]( https://github.com/ncbi/datasets/blob/workshop-cshl-2021/training/cshl-2021/jq_cheatsheet.md) to help you extract information from those files.  

## Part I: Accessing genomes<a class="anchor" id="Part-I"></a>

![workflow](./images/elmo_workflow.drawio.png)

First, let's figure out what kind of genome information NCBI has for ants (family Formicidae).

<img src="./images/genome_summary.drawio.png" style="width: 600px;"/>

In [2]:
%%bash
# Get metadata info - this example output has been truncated to fit the notebook output screen in github
# by printing only the first 1500 characters

datasets summary genome taxon formicidae | cut -c1-1500

# Original code run in the workshop
# datasets summary genome taxon formicidae


{"assemblies": [{"assembly": {"annotation_metadata":{"file":[{"estimated_size":"3421616","type":"GENOME_GFF"},{"estimated_size":"129483045","type":"GENOME_GBFF"},{"estimated_size":"3444924","type":"PROT_FASTA"},{"estimated_size":"2684704","type":"GENOME_GTF"},{"estimated_size":"7862131","type":"CDS_FASTA"}],"name":"From INSDC submitter","release_date":"2021-03-29","source":"BGI","stats":{"gene_counts":{"protein_coding":8986,"total":14640}}},"assembly_accession":"GCA_017607545.1","assembly_category":"representative genome","assembly_level":"Scaffold","bioproject_lineages":[{"bioprojects":[{"accession":"PRJNA605929","title":"Project of the leaf-cutting ants"}]}],"biosample_accession":"SAMN14167745","blast_url":"https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch\u0026PROG_DEF=blastn\u0026BLAST_SPEC=GDH_GCA_017607545.1","chromosomes":[{"length":"296539234","name":"Un"}],"contig_n50":34925,"display_name":"ASM1760754v1","estimated_size":"241200730","gc_count":"99149685","org":{"a

In [3]:
%%bash
# Get metadata info and save to a file
datasets summary genome taxon formicidae > formicidae_summary.json

**Now let's take a look at the metadata using jq**

In [4]:
%%bash
# this example has been truncated to fit the notebook output in github by only printing the first 100 lines
datasets summary genome taxon formicidae | jq . | perl -ne'1..100 and print' 

# Original code run in the workshop
# datasets summary genome taxon formicidae | jq . 

{
  "assemblies": [
    {
      "assembly": {
        "annotation_metadata": {
          "file": [
            {
              "estimated_size": "3421616",
              "type": "GENOME_GFF"
            },
            {
              "estimated_size": "129483045",
              "type": "GENOME_GBFF"
            },
            {
              "estimated_size": "3444924",
              "type": "PROT_FASTA"
            },
            {
              "estimated_size": "2684704",
              "type": "GENOME_GTF"
            },
            {
              "estimated_size": "7862131",
              "type": "CDS_FASTA"
            }
          ],
          "name": "From INSDC submitter",
          "release_date": "2021-03-29",
          "source": "BGI",
          "stats": {
            "gene_counts": {
              "protein_coding": 8986,
              "total": 14640
            }
          }
        },
        "assembly_accession": "GCA_017607545.1",
        "assembly_category": "representa

### A little bit more about json files
A JSON (JavaScript Object Notation) file stores data structures and objects. In a very simplified (and non-technical) way, a JSON file is a box, that might contain other boxes with more boxes inside. In `datasets summary genome` our JSON "box" is organized like this:
<img src="./images/json8.png" alt="image"/>

But let's explore the "boxes" in stages, so we can understand how everything is organized and how we can use this knowledge to extract information from the summary metadata file. At the first level, we have this: 
```
{
 assemblies[
      assembly{},
      assembly{},
 ],
 total_count
}
```
<img src="./images/json1.png" />

If we want to look at the value in the field "total_count", here's the command we would use:

In [5]:
%%bash
datasets summary genome taxon herpestidae | jq '.total_count'

5


If we continue to expand each one of those assembly boxes, more levels of the hierarchy will be revealed. Let's expand each assembly and look at what information we can find at that level.  

<img src="./images/json2a.png" alt="image"/>

Here we can see that some of the assembly information, such as assembly accession number, contig N50 or submission date are not included inside any of the available "boxes" (annotation_metadata, chromosomes, bioproject_lineage, and org). Those fields describe assembly features/characteristics that pertain to the entire assembly, and not only any of those boxes available. What are the contig n50 values of those assemblies?

To retrieve that information, we need to call each box, starting from the largest one, until the field we're interested in. And each level is separated from the next by a period (.). 

<img src="./images/json3.png" />

In [6]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly.contig_n50'

113567
180702
75409
148487
75409


Now let's see what we have inside `annotation_metadata`, `bioproject_lineages`, `org` and `chromosomes`. 
<img src="./images/json8.png" alt="image"/>

Now let's see how we can retrieve the scientific names associated with those assemblies.
<img src="./images/json4.png" />

In [7]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly.org.sci_name'

"Helogale parvula"
"Mungos mungo"
"Suricata suricatta"
"Suricata suricatta"
"Suricata suricatta"


As you can see, `jq` is very useful in retrieving information from the summary metadata *as long as* you know the path to find it. Let's try a few more complex examples.
<img src="./images/json5.png" />

First, let's retrieve information from three fields at the same time: scientific name (`sci_name`), assembly accession number (`assembly_accession`) and contig N50 (`contig_n50`).

In [8]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly | (.org.sci_name, .assembly_accession, .contig_n50)'

"Helogale parvula"
"GCA_004023845.1"
113567
"Mungos mungo"
"GCA_004023785.1"
180702
"Suricata suricatta"
"GCF_006229205.1"
75409
"Suricata suricatta"
"GCA_004023905.1"
148487
"Suricata suricatta"
"GCA_006229205.1"
75409


Since all three fields are inside the `.assemblies[].assembly`, we can call the first part of the path once and use a pipe (|) to call each specific field. 
Now let's try to make this a little easier to read. We can create new fields and assign values to them, like this:

In [9]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly 
| {species: .org.sci_name, accession: .assembly_accession, contigN50:.contig_n50}'

{
  "species": "Helogale parvula",
  "accession": "GCA_004023845.1",
  "contigN50": 113567
}
{
  "species": "Mungos mungo",
  "accession": "GCA_004023785.1",
  "contigN50": 180702
}
{
  "species": "Suricata suricatta",
  "accession": "GCF_006229205.1",
  "contigN50": 75409
}
{
  "species": "Suricata suricatta",
  "accession": "GCA_004023905.1",
  "contigN50": 148487
}
{
  "species": "Suricata suricatta",
  "accession": "GCA_006229205.1",
  "contigN50": 75409
}


**Last one**: let's look at a larger collection of genome assemblies (let's say, all Carnivora) and select only those assemblies with contig N50 larger than 15 Mb (15000000 bp). `datasets` provides many options for filtering, but there is no built-in filter for contig N50 size.  

Here's what we want to see: assembly accession number, species and assembly level for those genomes with contig N50 above 15 Mb.

In [10]:
%%bash
datasets summary genome taxon carnivora | jq -r '.assemblies[].assembly 
| select(.contig_n50 > 15000000) 
| [.assembly_accession, .org.sci_name, .assembly_level] 
| @tsv'

GCA_905319855.2	Canis lupus	Chromosome
GCF_012295265.1	Canis lupus dingo	Chromosome
GCA_003254725.2	Canis lupus dingo	Chromosome
GCA_012295265.1	Canis lupus dingo	Chromosome
GCF_000002285.5	Canis lupus familiaris	Chromosome
GCF_013276365.1	Canis lupus familiaris	Chromosome
GCA_000002285.4	Canis lupus familiaris	Chromosome
GCA_008641055.3	Canis lupus familiaris	Chromosome
GCA_013276365.2	Canis lupus familiaris	Chromosome
GCF_018350175.1	Felis catus	Chromosome
GCF_000181335.3	Felis catus	Chromosome
GCA_000181335.5	Felis catus	Chromosome
GCA_013340865.1	Felis catus	Contig
GCA_016509815.2	Felis catus	Chromosome
GCA_018350175.1	Felis catus	Chromosome
GCA_019924945.1	Felis chaus	Chromosome
GCF_018350155.1	Leopardus geoffroyi	Chromosome
GCA_018350155.1	Leopardus geoffroyi	Chromosome
GCA_902655055.2	Lutra lutra	Chromosome
GCA_922984935.1	Meles meles	Chromosome
GCF_009829155.1	Mustela erminea	Chromosome
GCA_009829155.1	Mustela erminea	Chromosome
GCF_011764305.1	Mustela putorius furo	Contig
GCA_

**RESOURCE:**  
We included a list of all fields in the genome summary in our [jq cheatsheet](https://github.com/ncbi/datasets/blob/workshop-cshl-2021/training/cshl-2021/jq_cheatsheet.md) to help you extract the information you need. And we will show you now how to do that. 

### Let's continue to explore the available genomes for the family Formicidae

<img src="./images/genome_summary.drawio.png" alt="summary" style="width: 600px"/>

For this part, we will use two UNIX commands: `sort` and `uniq`. 

- `sort` can be used to sort text files line by line, numerically and alphabetically.   
- `uniq` will filter out the repeated lines in a file. However, `uniq` can only detect repeated lines if they are adjacent to each other. In other words, if they are alphabetically or numerically sorted. The flag `-c` or `--count` tells the command `uniq` to remove the repeated lines, and to count how many times each value appeared. 

So, we will use `jq` to extract the information we need, sort the result and count the number of unique entries.

In [11]:
%%bash
# For which species does NCBI have genomes in its database? How many per species?

datasets summary genome taxon formicidae | jq '.assemblies[].assembly.org.sci_name' | sort | uniq -c

   1 "Acromyrmex charruanus"
   2 "Acromyrmex echinatior"
   1 "Acromyrmex heyeri"
   1 "Acromyrmex insinuator"
   1 "Aphaenogaster ashmeadi"
   1 "Aphaenogaster floridana"
   1 "Aphaenogaster fulva"
   1 "Aphaenogaster miamiana"
   1 "Aphaenogaster picea"
   2 "Aphaenogaster rudis"
   2 "Atta cephalotes"
   2 "Atta colombica"
   1 "Atta texana"
   4 "Camponotus floridanus"
   1 "Cardiocondyla obscurior"
   1 "Cataglyphis hispanica"
   1 "Cataglyphis niger"
   1 "Crematogaster levior"
   2 "Cyphomyrmex costatus"
   2 "Dinoponera quadriceps"
   1 "Eciton burchellii"
   1 "Formica aquilonia x Formica polyctena"
   2 "Formica exsecta"
   1 "Formica selysi"
   4 "Harpegnathos saltator"
   1 "Lasius niger"
   2 "Linepithema humile"
   7 "Monomorium pharaonis"
   2 "Nylanderia fulva"
   2 "Odontomachus brunneus"
   4 "Ooceraea biroi"
   2 "Pogonomyrmex barbatus"
   1 "Pogonomyrmex californicus"
   1 "Pseudoatta argentina"
   1 "Pseudomyrmex concolor"
   1 "Pseudomyrmex cubaensis"
   1 "Pseud

In [12]:
%%bash
# What is the assembly level (contig, scaffold, chromosome, complete) breakdown?

datasets summary genome taxon formicidae | jq '.assemblies[].assembly.assembly_level' | sort | uniq -c

  12 "Chromosome"
   1 "Contig"
  84 "Scaffold"


### How to get help when using the command line

Since `datasets` is a very hierarchical program, we can use that characteristic to our advantage to get very specific help.   For example: if we type `datasets --help`, we will see the first level of commands available.


In [13]:
%%bash
datasets --help

datasets is a command-line tool that is used to query and download biological sequence data
across all domains of life from NCBI databases.

Refer to NCBI's [command line quickstart](https://www.ncbi.nlm.nih.gov/datasets/docs/quickstarts/command-line-tools/) documentation for information about getting started with the command-line tools.

Usage
  datasets [command]

Data Retrieval Commands
  summary              print a summary of a gene or genome dataset
  download             download a gene, genome or coronavirus dataset as a zip file
  rehydrate            rehydrate a downloaded, dehydrated dataset

Miscellaneous Commands
  completion           generate autocompletion scripts
  version              print the version of this client and exit
  help                 Help about any command

Flags
      --api-key string   NCBI Datasets API Key
  -h, --help             help for datasets
      --no-progressbar   hide progress bar

Use datasets help <command> for detailed help about a comma

Notice the difference from when we type `datasets summary genome taxon formicidae --help`  


In [14]:
%%bash
datasets summary genome taxon formicidae --help


Print a summary of a genome dataset by taxon (NCBI Taxonomy ID, scientific or common name at any tax rank). The summary is returned in JSON format.

Refer to NCBI's [command line quickstart](https://www.ncbi.nlm.nih.gov/datasets/docs/quickstarts/command-line-tools/) documentation for information about getting started with the command-line tools.

Usage
  datasets summary genome taxon [flags]

Examples
  datasets summary genome taxon human
  datasets summary genome taxon "mus musculus"
  datasets summary genome taxon 10116

Flags
  -h, --help              help for taxon
      --tax-exact-match   exclude sub-species when a species-level taxon is specified


Global Flags
  -a, --annotated                only include genomes with annotation
      --api-key string           NCBI Datasets API Key
      --as-json-lines            Stream results as newline delimited JSON-Lines
      --assembly-level string    restrict assemblies to a comma-separated list of one or more of: chromosome, complet

### Exercises

Now we will practice what we learned about `datasets`. Take a look at the questions below and feel free to ask questions. Useful resources for this exercise are the `--help` from the command line and the [jq cheatsheet](). 


In [15]:
%%bash
# How many reference genomes in the family Formicidae? (hint --reference)



In [16]:
%%bash
# How many reference genomes are annotated? (hint: --annotated)



In [17]:
%%bash
# How many genomes have NCBI (RefSeq) annotations? (hint: --assembly-source)



### Bonus questions:

In [18]:
%%bash
## Take a look at the jq cheat sheet (link here) and try to build a jq query for the metadata



In [19]:
%%bash
# Now look at the summary metadata for your organism of interest 
# (if you don't have a favorite, go with red panda, Ailurus fulgens, taxid: 9649)



In [20]:
%%bash
# How many genomes?



In [21]:
%%bash
# Assembly level breakdown



In [22]:
%%bash
# How many genomes have a contig N50 value above 15Mb?



### Back to the main room


### What is the difference/relationship between Genbank, RefSeq and Reference assemblies?

<img src="./images/gca_gcf.png" alt="ref" />

### Data package

We explored the `datasets summary` option, in which we had a chance to look at the summary metadata ***without*** downloading any files. In the next steps, we will look at the data packages, which contain the actual data files. 

<img src="./images/genome_data_package.png" alt="data_package" />

In [23]:
%%bash
# Download all available GenBank assemblies for the genus Acromyrmex and save as genomes.zip
datasets download genome taxon acromyrmex --assembly-source genbank --filename genomes.zip --no-progressbar

In [24]:
%%bash
# Unzip genomes.zip to the folder genomes
unzip -o genomes.zip -d genomes

Archive:  genomes.zip
  inflating: genomes/README.md       
  inflating: genomes/ncbi_dataset/data/assembly_data_report.jsonl  
  inflating: genomes/ncbi_dataset/data/GCA_000204515.1/unplaced.scaf.fna  
  inflating: genomes/ncbi_dataset/data/GCA_000204515.1/cds_from_genomic.fna  
  inflating: genomes/ncbi_dataset/data/GCA_000204515.1/genomic.gff  
  inflating: genomes/ncbi_dataset/data/GCA_000204515.1/protein.faa  
  inflating: genomes/ncbi_dataset/data/GCA_017607455.1/GCA_017607455.1_ASM1760745v1_genomic.fna  
  inflating: genomes/ncbi_dataset/data/GCA_017607455.1/cds_from_genomic.fna  
  inflating: genomes/ncbi_dataset/data/GCA_017607455.1/genomic.gff  
  inflating: genomes/ncbi_dataset/data/GCA_017607455.1/protein.faa  
  inflating: genomes/ncbi_dataset/data/GCA_017607545.1/GCA_017607545.1_ASM1760754v1_genomic.fna  
  inflating: genomes/ncbi_dataset/data/GCA_017607545.1/cds_from_genomic.fna  
  inflating: genomes/ncbi_dataset/data/GCA_017607545.1/genomic.gff  
  inflating: genomes/n

In [25]:
%%bash
# Explore the folder structure of the folder genome with the command tree
tree -C genomes/

genomes/
├── README.md
└── ncbi_dataset
    └── data
        ├── GCA_000204515.1
        │   ├── cds_from_genomic.fna
        │   ├── genomic.gff
        │   ├── protein.faa
        │   ├── sequence_report.jsonl
        │   └── unplaced.scaf.fna
        ├── GCA_017607455.1
        │   ├── GCA_017607455.1_ASM1760745v1_genomic.fna
        │   ├── cds_from_genomic.fna
        │   ├── genomic.gff
        │   ├── protein.faa
        │   └── sequence_report.jsonl
        ├── GCA_017607545.1
        │   ├── GCA_017607545.1_ASM1760754v1_genomic.fna
        │   ├── cds_from_genomic.fna
        │   ├── genomic.gff
        │   └── protein.faa
        ├── GCA_017607565.1
        │   ├── GCA_017607565.1_ASM1760756v1_genomic.fna
        │   ├── cds_from_genomic.fna
        │   ├── genomic.gff
        │   └── protein.faa
        ├── assembly_data_report.jsonl
        └── dataset_catalog.json

6 directories, 21 files


### Let's recap our goals

We used `datasets` to download all the Genbank assemblies for the genus *Acromyrmex*. The next step is to download the gene *orco* (odorance receptor coreceptor) for the same genus. But first, let's learn more about how genes are organized at NCBI.

<img src="./images/elmo_done1.png" alt="done1" style="width: 500px;" />